<h1 style="color:red;"><strong>1. Look at the big Picture</h1>

## **1.1 Frame the Problems**

* **What's the business objective**
* **How does the company expect to use and benefit from the model?**

## **1.2 Chose performance measure (Loss)**


### **1.2.1 Regression**
* RMSE
* MAE
* MSE, 
* RMSLE
* MAPE
* SMAPE

### **1.2.2 Classification**
* Binary Cross Entropy 
* Sparse Categorical Cross Entropy
* Categorial Cross Entropy

## **1.3 Chose performance measure (Metrics)**

### **1.3.1 Regression**
* Adjusted r^2

### **1.3.2 Classification**
* AUC
* Accuracy
* False Positive
* False Negative
* F1 Score
* Confusion matrix

## **1.4 Check the Assumptions** 

<h1 style="color:red;"><strong>2. Get the Data</h1>

<h1 style="color:red;"><strong>3. Data Cleaning (General)</h1>

## **3.1 Formating**

### **3.1.1 Format column names**

### **3.1.2 Format data body**

### **3.1.3 Format Datatypes**

## **3.2 Drop duplicates rows**

<h1 style="color:red;"><strong>4. Attribute Combination</h1>

<h1 style="color:red;"><strong> 5. EDA</h1>

## **5.1 Univariate**

## **5.2 Bivariate**

## **5.3 Multivariate**

<h1 style="color:red;"><strong> 6. Labels Cleaning for Modelling (Optionals)</h1>

* Drop null labels
* Drop outlier labels

<h1 style="color:red;"><strong> 7. Split Train Test For Modelling </h1>

<h1 style="color:red;"><strong> 8. Data Cleaning for Modelling </h1>

## **8.1 Handle Nulls**

### 8.1.1 Types of NULLS

**Missing Completely at Random (MCAR)**
- Missing data is random
- Data was lost in ETL, someone was interrupted when completing a survey
- Remove or impute

**Missing at Random (MAR)**
- Missing data suggests something about something else observed
- Older (which we have in data) may have higher privacy concerns and not report income
- Remove or impute

**Missing Not at Random (MNAR)**
- Missing based on something not observed
- Self-selection bias: Depressed do not complete mental health surveys
- Advanced econometrics

**By Design:**
- Remove credit card data if birthdate is below a certain value

### 8.1.2 Solutions to handle Nulls 
- Get rid of the corresponding rows.
- Get rid of the whole attribute.
- Set the values to some value (zero, the mean, the median, etc.).
- Create a new column that have True for the non-null and False for null (or the opposite)

Fit train (non_labels)

Fit Labels

## **8.2 Handle Outliers**

<h1 style="color:red;"><strong> 9.Preprocess Data for Modelling </h1>

## **9.1 Handle Categorical Data**

### 9.1.1 Ordinal Encoding

### 9.1.2 One Hot Encoding

## **9.2 Feature Scaling**
* As with all the transformations, it is important to fit the scalers to the training data only, not to the full dataset (including the test set). Only then can you use them to transform the training set and the test set (and new data).

### 9.2.1 Min-Max Scailing (Normalization Scailing)
* Values are shifted and rescaled so that they end up ranging from 0 to 1. We do this by subtracting the min value and dividing by the max minus the min
* Normalization are more affected by outliers

### 9.2.2 Standardization Scailing 
* First it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the standard deviation so that the resulting distribution has unit variance
*  Unlike min-max scaling, standardization does not bound values to a specific range, which may be a problem for some algorithms (e.g., neural networks often expect an input value ranging from 0 to 1)
* standardization is much less affected by outliers.

## **9.3 Transformation Pipelines**

<h1 style="color:red;"><strong> 10. Select and Train a Model </h1>

## **10.1 Training and Evaluating on the Training Set**

### 10.1.1 Model 1: Linear Regression Model

### 10.1.2 Model 2: Decision Tree Regressor Model

### 10.1.3 Model 3: Forest Regressor Model

Could this model really be absolutely perfect? Of course,
it is much more likely that the model has badly overfit the data. How can you be sure?
As we saw earlier, you don’t want to touch the test set until you are ready to launch a
model you are confident about, so you need to use part of the training set for train‐
ing, and part for model validation

## **10.2 Better Evaluation Using Cross-Validation**

The score on the training set is still much lower than on the validation sets, meaning that the model is still overfitting the training set. The solution include:
* Simplify the model
* Constrain it (i.e., regularize it)
* Get a lot more training data

Try others ML models, the goal is to shortlist 2-5 promising models

## **11.1 Grid Search**
One way to do that would be to fiddle with the hyperparameters manually, until you find a great combination of hyperparameter values. This would be very tedious work, and you may not have time to explore many combinations.


Instead you should get Scikit-Learn’s GridSearchCV to search for you. All you need to do is tell it which hyperparameters you want it to experiment with, and what values to try out, and it will evaluate all the possible combinations of hyperparameter values, using cross-validation.

## **11.2 Randomized Search**
Better when the hyperparameter space is large. This approach is the same way as the GridSearchCV class, but instead of trying out all possible combinations, it evaluates a given number of random combinations by selecting a random value for each hyperparameter at every iteration. This approach has two main benefits:
* If you let the randomized search run for, say, 1,000 iterations, this approach will explore 1,000 different values for each hyperparameter (instead of just a few values per hyperparameter with the grid search approach).
* You have more control over the computing budget you want to allocate to hyperparameter search, simply by setting the number of iterations


## **11.3 Ensemble Methods**
Another way to fine-tune your system is to try to combine the models that perform best. The group (or “ensemble”) will often perform better than the best individual model (just like Random Forests perform better than the individual Decision Trees they rely on), especially if the individual models make very different types of errors.

<h1 style="color:red;"><strong>   12. Analyze Model and Explain Features important </h1>

## **12.1 Grid Search Feature Importance**

## **12.2 Lime**

## **12.3 Shapley**

<h1 style="color:red;"><strong>   13 Evaluate Your System on the Test Set </h1>

Now is the time to evaluate the final model on the test set. There is nothing special about this process; just get the predictors and the labels from your test set, run your full_pipeline to transform the data (call `transform`, not `fit_transform()`, you do not want to fit the test set!), and evaluate the final model on the test set:
